In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from pixelmed_calc.medical_imaging.modelstore import get_model_from_name
model=get_model_from_name['resnet50'](num_classes = 2, pretrained = False,in_channels=3).to("cuda")
#如果有预训练好的2D模型，则修改以下代码路径
#model.load_state_dict(torch.load(r'd:\pixelmedAI\function_pm\note2-深度学习分类\results\latest_model.pth'))
from pixelmed_calc.medical_imaging.DLtools.process2d import load_data_from_csv,get_file_and_labels,get_transforms,CustomImageDataset,train_and_validate
import torch
from monai.data import DataLoader
train_base_path = r'D:\pixelmedAI\function_pm\pm_data\MR_jpg\images'
val_base_path = r'D:\pixelmedAI\function_pm\pm_data\MR_jpg\images'
input_size=224
# CSV文件路径
csv_path = r'D:\pixelmedAI\function_pm\pm_data\MR_jpg\group.csv'
tfs =get_transforms(input_size)

# 读取所有数据
file_names, labels, groups = load_data_from_csv(csv_path)

# 获取训练数据和标签（可以设置 base_path 为 None 或实际路径）
file_train, train_labels = get_file_and_labels(file_names, labels, groups, 'train', train_base_path)

# 获取验证数据和标签（可以设置 base_path 为 None 或实际路径）
file_val, val_labels = get_file_and_labels(file_names, labels, groups, 'test', val_base_path)

# 假设你已经定义了 ImageDataset 和 train_transforms, val_transforms
train_ds = CustomImageDataset(image_files=file_train, labels=train_labels, transform=tfs)
train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=0, pin_memory=torch.cuda.is_available())

val_ds = CustomImageDataset(image_files=file_val, labels=val_labels, transform=tfs)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=0, pin_memory=torch.cuda.is_available())



In [2]:
for name, _ in model.named_modules(): print(name)


conv1
bn1
relu
maxpool
layer1
layer1.0
layer1.0.conv1
layer1.0.bn1
layer1.0.conv2
layer1.0.bn2
layer1.0.conv3
layer1.0.bn3
layer1.0.relu
layer1.0.downsample
layer1.0.downsample.0
layer1.0.downsample.1
layer1.1
layer1.1.conv1
layer1.1.bn1
layer1.1.conv2
layer1.1.bn2
layer1.1.conv3
layer1.1.bn3
layer1.1.relu
layer1.2
layer1.2.conv1
layer1.2.bn1
layer1.2.conv2
layer1.2.bn2
layer1.2.conv3
layer1.2.bn3
layer1.2.relu
layer2
layer2.0
layer2.0.conv1
layer2.0.bn1
layer2.0.conv2
layer2.0.bn2
layer2.0.conv3
layer2.0.bn3
layer2.0.relu
layer2.0.downsample
layer2.0.downsample.0
layer2.0.downsample.1
layer2.1
layer2.1.conv1
layer2.1.bn1
layer2.1.conv2
layer2.1.bn2
layer2.1.conv3
layer2.1.bn3
layer2.1.relu
layer2.2
layer2.2.conv1
layer2.2.bn1
layer2.2.conv2
layer2.2.bn2
layer2.2.conv3
layer2.2.bn3
layer2.2.relu
layer2.3
layer2.3.conv1
layer2.3.bn1
layer2.3.conv2
layer2.3.bn2
layer2.3.conv3
layer2.3.bn3
layer2.3.relu
layer3
layer3.0
layer3.0.conv1
layer3.0.bn1
layer3.0.conv2
layer3.0.bn2
layer3.0.conv

In [3]:
import numpy as np
import pandas as pd
from pixelmed_calc.medical_imaging.DLtools.utils import extract_layer_output
val_ds_viz = CustomImageDataset(image_files=file_val, labels=val_labels,transform=tfs)  
val_loader_viz = DataLoader(val_ds_viz, batch_size=1, shuffle=False, num_workers=0, pin_memory=torch.cuda.is_available())  

DL_output=pd.DataFrame()
i=0
for sample in val_loader: 
        ids = val_loader.dataset.image_files[i]
        avg_pool_output = extract_layer_output(model, sample[0], 'avgpool')  
        avg_pool_output=avg_pool_output.cpu().detach().numpy()
        avg_pool_output=avg_pool_output.reshape(1,-1)
        ids_df = pd.Series(ids)  
        avg_pool_output_df = pd.DataFrame(avg_pool_output)  
        # Concatenate ids and avg_pool_output horizontally  
        combined_df = pd.concat([ids_df.reset_index(drop=True), avg_pool_output_df.reset_index(drop=True)], axis=1)  
        # Append the combined DataFrame to DL_output  
        DL_output = pd.concat([DL_output, combined_df], ignore_index=True) 
        i=i+1

DL_output


The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).


,0,0,1,2,3,4,5,6,7,8,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,D:\pixelmedAI\function_pm\pm_data\MR_jpg\image...,1.005716,1.069668,0.981972,0.996226,1.055698,0.965523,1.083309,0.980493,1.070751,...,1.085962,1.106277,1.047822,1.071735,1.018887,0.969021,0.933900,1.151250,0.918495,1.075510
1,D:\pixelmedAI\function_pm\pm_data\MR_jpg\image...,0.992333,1.049175,0.915082,1.095711,0.937805,0.959354,1.047274,0.946614,1.027452,...,1.090970,1.063175,1.034309,0.983135,0.928482,1.098796,1.104253,1.021731,1.078084,1.112102
2,D:\pixelmedAI\function_pm\pm_data\MR_jpg\image...,0.995550,0.957909,0.967934,1.001516,0.945011,0.976077,1.129734,0.958531,1.026139,...,1.047215,1.012561,0.909573,1.002119,0.927284,1.042491,1.026200,1.074102,1.038380,1.032816
3,D:\pixelmedAI\function_pm\pm_data\MR_jpg\image...,1.055863,1.111578,1.005450,1.066006,1.014837,0.978016,0.967966,1.038974,1.086278,...,1.051257,0.935405,0.972072,1.002953,0.994002,1.038411,1.086414,1.106777,0.998857,1.104562
4,D:\pixelmedAI\function_pm\pm_data\MR_jpg\image...,1.105017,1.021188,1.005870,0.869819,1.035794,0.998619,1.075453,1.075262,1.043473,...,1.039047,1.058686,1.073330,1.083738,0.980277,0.966072,0.999589,1.120003,1.033813,0.981021
5,D:\pixelmedAI\function_pm\pm_data\MR_jpg\image...,0.938348,0.943282,1.019217,1.012228,1.055869,0.920827,1.116844,0.931545,0.892106,...,0.922866,1.182310,0.936789,1.088931,0.833010,0.949339,1.129775,1.096918,0.957832,1.164439
6,D:\pixelmedAI\function_pm\pm_data\MR_jpg\image...,1.044341,1.113209,0.968424,1.088917,1.087553,1.064455,1.119092,0.982685,0.941111,...,1.152188,1.149870,1.110666,1.071458,1.010503,1.079121,0.989875,1.007218,1.085383,1.003948


In [9]:
DL_output.to_csv('DL_feature.csv',index=False)